### Passo 1 - Importar Arquivos e Bibliotecas

In [ ]:
# Importar as bibliotecas

import pandas as pd
import pathlib
import win32com.client as win32

In [ ]:
# Importar as Bases de Dados

emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
display(emails.head())

lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
display(lojas.head())

vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
display(vendas.head())

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [ ]:
# Incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')
# display(vendas)

In [ ]:
# Criar um dicionário
dicionario_lojas = {}

for loja in lojas['Loja']:
    # Criando as lojas como chaves dentro do dicionário
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :] # dois pontos para mostrar que pegamos todas as colunas
                                # .loc[linha, coluna]

# display(dicionario_lojas['Rio Mar Recife'])
# display(dicionario_lojas['Shopping Vila Velha'])
display(dicionario_lojas['Shopping Eldorado'])

In [ ]:
# Definindo o Dia do Indicador (temos essa informação na base de vendas)

dia_do_indicador = vendas['Data'].max() # pegando o dia mais atual, ou seja, o maior, mais atual
print(dia_do_indicador)
print('{}/{}'.format(dia_do_indicador.day, dia_do_indicador.month ))

### Passo 3 - Salvar a planilha na pasta de backup

In [ ]:
# Identificar se a pasta ja existe - caso exista não irá criar, apenas adicionar

import pathlib

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir() # o iterdir pega todos os objetos que estão dentro da pasta

lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup] # USANDO O LIST COMPREHENSION

# for arquivo in arquivos_pasta_backup: # USANDO O FOR
#     lista_nomes_backup.append(arquivo.name)

for loja in dicionario_lojas:
    # print(loja)
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja # como ja criamos o caminho pasta colocarmos o barra (/) nome do arquivo
        nova_pasta.mkdir() # serve para importar as pastas para o diretório
    
    # Salvar dentro da pasta

    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_do_indicador.month, dia_do_indicador.day, loja)
    # print(nome_arquivo)
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo, index = False)

### Passo 4 - Calcular o indicador para 1 loja

In [ ]:
# Definição de metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtde_produtos_dia = 4
meta_qtde_produtos_ano = 120
meta_ticket_medio_dia = 500
meta_ticket_medio_ano = 500

In [ ]:
for loja in dicionario_lojas:
    vendas_loja = dicionario_lojas[loja]

    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_do_indicador, :]

    # faturamento - somar coluna do valor final de cada loja
    faturamento_ano = vendas_loja['Valor Final'].sum()
    # print(faturamento_ano)

    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    # print(faturamento_dia)

    # diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique()) #método 'unique' tira os valores duplicados
    # print(qtde_produtos_ano)

    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    # print(qtde_produtos_dia)

    # ticket médio

    # ticket médio ano
    valor_venda = vendas_loja.groupby('Código Venda').sum() # agrupando por
    # display(valor_venda)
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    # print(ticket_medio_ano)

    # ticket médio dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean() # pegando a média da soma total
    # print(ticket_medio_dia)

    #Enviado e-mails
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja'] == loja ,'Gerente'].values[0]

    mail = outlook.CreateItem(0)
    # Enviar para a pessoa que está na mesma linha que o nome da loja
    mail.To = emails.loc[emails['Loja'] == loja ,'E-mail'].values[0] # faz com que pegemos o valor em si, não uma tabela com índice
    # mail.CC = 
    # mail.BCC =
    mail.subject = f'One Page Dia {dia_do_indicador.day}/{dia_do_indicador.month} - Loja {loja}'

    if faturamento_dia >= meta_ticket_medio_dia:
        cor_fat_dia = "green"
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtde_produtos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtde_produtos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticket_medio_dia:
        cor_ticket_dia = "green"
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticket_medio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    # Enviar o E-MAIL
    #ou mail.Body = ' Corpo do e-mail normal '
    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>
    <p>O resultado de ontem <strong>({dia_do_indicador.day}/{dia_do_indicador.month})</strong> da <strong>Loja {loja}</strong> foi: </p>

    <table> 
        <tr>
        
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${faturamento_dia:.2f}</td>
            <td style="text-align: center">R${meta_faturamento_dia}</td>
            <td style="text-align: center"><font color='{cor_fat_dia}'>◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{qtde_produtos_dia}</td>
            <td style="text-align: center">{meta_qtde_produtos_dia}</td>
            <td style="text-align: center"><font color='{cor_qtde_dia}'>◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
            <td style="text-align: center">R${meta_ticket_medio_dia:.2f}</td>
            <td style="text-align: center"><font color='{cor_ticket_dia}'>◙</font></td>
        </tr>
    </table>
    <br>
    <table> 
        <tr>
            <th>Indicador</th>
            <th>Valor Ano</th>
            <th>Meta Ano</th>
            <th>Cenário Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${faturamento_ano:.2f}</td>
            <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
            <td style="text-align: center"><font color='{cor_fat_ano}'>◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{qtde_produtos_ano}</td>
            <td style="text-align: center">{meta_qtde_produtos_ano}</td>
            <td style="text-align: center"><font color='{cor_qtde_ano}'>◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
            <td style="text-align: center">R${meta_ticket_medio_ano:.2f}</td>
            <td style="text-align: center"><font color='{cor_ticket_ano}'>◙</font></td>
        </tr>
    </table>

    <p>Segue em anexo a planilha comtodos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou á disposição</p>
    <p>Att., Lucas</p>
    '''
    #ou mail.Body = ' Corpo do e-mail normal '

    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_do_indicador.month}_{dia_do_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.send
    print(f'E-mail da loja {loja} enviado ')


### Passo 5 - Enviar por e-mail para o gerente

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [29]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending= False)
display(faturamento_lojas_ano)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_do_indicador.month, dia_do_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

                        #linha, coluna
vendas_dia = vendas.loc[vendas['Data'] == dia_do_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending= False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_do_indicador.month, dia_do_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

C:\Users\lucas\AppData\Local\Temp\ipykernel_23340\3913238946.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()


,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


C:\Users\lucas\AppData\Local\Temp\ipykernel_23340\3913238946.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Passo 8 - Enviar e-mail para diretoria

In [31]:
#Enviado e-mails
outlook = win32.Dispatch('outlook.application')

nome = emails.loc[emails['Loja'] == loja ,'Gerente'].values[0]

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja'] == 'Diretoria' ,'E-mail'].values[0] # faz com que pegemos o valor em si, não uma tabela com índice
mail.subject = f'Ranking Dia {dia_do_indicador.day}/{dia_do_indicador.month}'

mail.Body = f''' 
Prezados,bom dia

Melhor loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[0]} com faturamento R${faturamento_lojas_dia.iloc[0,0]:.2f}
Pior loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[-1]} com faturamento R${faturamento_lojas_dia.iloc[-1,0]:.2f}

Melhor loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com faturamento R${faturamento_lojas_ano.iloc[0,0]:.2f}
Pior loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[-1]} com faturamento R${faturamento_lojas_ano.iloc[-1,0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou á disposição.

Att,
Lucas
 '''

#ou mail.Body = ' Corpo do e-mail normal '

attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_do_indicador.month}_{dia_do_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))

attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_do_indicador.month}_{dia_do_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.send
print(f'E-mail da Diretoria enviado ')


E-mail da Diretoria enviado 
